In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import pathlib

import seaborn as sns
from matplotlib import pyplot as plt
import networkx as nx

import navis 
import navis.interfaces.neuprint as neu
import neuprint as neu_orig
import NeuromodPlasticity as nmp 


plt.rcParams['pdf.fonttype']=42

%load_ext autoreload
%autoreload 2 


%matplotlib inline

In [2]:
# load eb coords



In [ ]:
# load example ER4d, EPG and an EL that connects them

neu.fetch_skeleton(with_distances=True)


In [ ]:
# get synapses from ER4d to EPG and EL to ER4d

# attach synapses to skeleton
neu.skeleton.upsample_skeleton
neu.skeleton.attach_synapses_to_skeleton
    # add rows to to dataframe, make parent node the closest node


In [ ]:
# for each ER-EPG synapse, calculate distance to nearest EL-Er synapse
neu.skeleton.skeleton_df_to_nx
nx.shortest_path_length